# Utility Notebook Voor Datasets
Dit notebook is specifiek bedoeld om foto's uit de CCPD2019 dataset (https://github.com/detectRecog/CCPD) om te zetten naar xml bestanden. 
De foto's worden tegelijkertijd hernoemd. 

In [4]:
import os
import re
import cv2
from xml.etree import cElementTree as ElementTree

Selecteer alle foto's uit het CCPD2019 dataset die je wilt gebruiken en verplaats deze naar de map 'TempFotos' (te vinden in Dataset\TempDataSet). 

In [5]:
root_path = r'../Dataset/TempDataSet/'
file_name = os.listdir(root_path)
class_name = 'plate'
xml_dir = root_path+"/TempAnnots"  # The directory where xml is saved

In [6]:
for i in file_name:
    file_name1 = os.listdir(os.path.join(root_path,i))
    counter = 0 # change this number if you are increasing your dataset. if you're adding 100 extra foto's to a datset of 50,
    # then change the number to start at 50. 
    for ii in file_name1:
        counter += 1 
        img = cv2.imread((root_path+i+'/'+ii))
        height = img.shape[0]
        width = img.shape[1]
        depth = img.shape[2]
        point = ii.split('.')[0].split('-')[3]  #
        num = re.findall('\d+\d*',point) # Regular expression to extract values from a string
        Xmin =min(num[0::2])  #　list[start:stop:step]
        Ymin = min(num[1::2])
        Xmax = max(num[0::2])
        Ymax = max(num[1::2])
        fname = ii.split('&')[0] + '&amp;'+ii.split('&')[1]+ '&amp;'+ii.split('&')[2]+ '&amp;'+ii.split('&')[3]+ '&amp;'+ii.split('&')[4]+ '&amp;'+ii.split('&')[5]+ '&amp;'+ii.split('&')[1]+ '&amp;'+ii.split('&')[6]
        new_name = str(counter).zfill(5)
        xml_str = "<annotation>\n\t\
                <folder>"+ i+ "</folder>\n\t\
                <filename>" + new_name +'.jpg' + "</filename>\n\t\
                " + "<path>" + r"./Dataset/images/"+new_name+'.jpg'+ "</path>\n\t\
                <source>\n\t\t\
                <database>Unknown</database>\n\t\
                </source>\n\t\
                <size>\n\t\t\
                <width>" + str(width) + "</width>\n\t\t\
                <height>" + str(height) + "</height>\n\t\t\
                <depth>" + str(depth) + "</depth>\n\t\
                </size>\n\t\
                <segmented>0</segmented>"
        obj_str = "\n\t\
                    <object>\n\t\t\
                    <name>" + class_name + "</name>\n\t\t\
                    <pose>Unspecified</pose>\n\t\t\
                    <truncated>0</truncated>\n\t\t\
                    <difficult>0</difficult>\n\t\t\
                    <bndbox>\n\t\t\t\
                    <xmin>" + str(Xmin) + "</xmin>\n\t\t\t\
                    <ymin>" + str(Ymin) + "</ymin>\n\t\t\t\
                    <xmax>" + str(Xmax) + "</xmax>\n\t\t\t\
                    <ymax>" + str(Ymax) + "</ymax>\n\t\t\
                    </bndbox>\n\t\
                    </object>"
        xml_str += obj_str
        xml_str +="\n</annotation>\n"
        #print(ii.split('.')[0]+'.xml')
        with open(xml_dir + '/'+new_name+'.xml','w') as f:
            f.write(xml_str)
            #print('Read and write successfully')
            f.close()
        src = r'../Dataset/TempDataSet/TempFotos/'+ii
        pic_name = r'../Dataset/TempDataSet/TempFotos/'+new_name+'.jpg'
        os.rename(src, pic_name)
print('end')

# source: https://www.programmersought.com/article/22976129791/

end


# Error Detectie
De CCPD2019 blijkt fouten te hebben in de x-max en x-min waardes waarbij de x-min soms groter is dan de x-max. De onderstaande code is bedoeld om xml files te vinden met deze fout. De gebruiker moet helaas handmatig de fouten er uit halen. 

In [9]:
class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

# source: https://code.activestate.com/recipes/410469-xml-as-dictionary/

In [10]:
path = "../Dataset/TempDataSet/TempAnnots/"
files = os.listdir(path)
for index, file in enumerate(files):
    tree = ElementTree.parse(path+file)
    root = tree.getroot()
    xmldict = XmlDictConfig(root)

    xmin = int(xmldict['object']['bndbox']['xmin'])
    xmax = int(xmldict['object']['bndbox']['xmax'])

    ymin = int(xmldict['object']['bndbox']['ymin'])
    ymax = int(xmldict['object']['bndbox']['ymax'])

    if xmin > xmax:
        print("x", file, xmin, xmax)

    if ymin > ymax:
        print("y", file, ymin, ymax)

Zodra alle files op orde zijn kan je de inhoud van TempFotos en TempAnnots verplaatsen naar TrainAndTest/images voor de fotos en TrainAndTest/annots voor de xmls. 

# Bronnen
McGreggor, D. M. G. (2005, April 19). XML as Dictionary « Python recipes « ActiveState Code. ActiveState. https://code.activestate.com/recipes/410469-xml-as-dictionary/

Generate annotation file (xml) from CCPD data set - Programmer Sought. (n.d.). ProgrammerSought. https://www.programmersought.com/article/22976129791/